# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」に参加するために設計されており、2つの異なる言語モデルの応答の好ましさを予測する問題に取り組んでいます。

## 問題へのアプローチ
Notebookでは、CatBoostを用いて、チャットボットの応答の優劣を予測するモデルを構築します。このモデルにより、2つの異なるチャットボットの応答の中で、どちらがユーザーによって好まれるかを確率として出力します。データセットには、与えられたプロンプトに対してモデルが生成した複数の応答と、それに対するユーザーの選好が含まれています。

## 使用されている手法とライブラリ
1. **ライブラリ**:
   - **bitsandbytes**および**transformers**: 大規模言語モデルを効率的に扱うためのライブラリとして使用されています。
   - **CatBoost**: 決定木を基にした機械学習アルゴリズムであり、特にカテゴリカルデータに強いモデルを構築します。
   - **PyTorch**: 深層学習モデルやテンソル操作を行うためのフレームワークです。
   - **Pandas**および**NumPy**: データ操作や数値計算に使用されます。

2. **主な手順**:
   - 必要なライブラリのインストールとインポート。
   - 訓練データの読み込みと埋め込み特徴の取得。
   - ターゲット変数の整備と訓練データとテストデータの分割。
   - CatBoostClassifierのモデルをロードし、予測を行う準備。
   - Gemmaモデルの初期化とトークナイザーの設定。
   - テストデータの整形およびトークン化を行い、モデルに入力するための形式に変換。
   - スレッドを使用して埋め込みを計算し、最終的にCatBoostモデルを使って予測確率を取得。
   - 提出用のCSVファイルを生成するために、予測結果をサンプルデータフレームに設定し保存。

このNotebookは、最終的に提出用ファイル「submission.csv」を生成し、コンペティションで使用する予測確率を出力します。この一連の処理を通じて、2つのチャットボットの応答に対して、ユーザーの好みを予測するモデルを効果的に構築しています。

---


# 用語概説 
以下に、jupyter notebookの内容をもとに、機械学習・深層学習の初心者がつまずきそうな専門用語の解説を列挙します。

1. **bitsandbytes**:
   - 深層学習モデルの量子化や圧縮に特化したライブラリ。メモリ使用量を削減し、計算を高速化することができる。特に大規模なモデルを扱う際に役立つ。

2. **トークナイザー**:
   - テキストデータをモデルが理解できる形（トークン）に変換するためのツール。単語やサブワードに分割し、内部的なIDに変換する。

3. **埋め込み（embedding）**:
   - データ（通常はテキスト）を連続的な数値ベクトルに変換する手法。これにより、データのセマンティックな関係が反映される。例えば、単語をベクトル空間にマッピングし、意味的に類似した単語が近くに配置される。

4. **CatBoost**:
   - 決定木に基づく勾配ブースティングアルゴリズム。特にカテゴリカルデータを扱うのに強く、高速で効果的なモデルを構築できる。デフォルトでオーバーフィッティングを避けるように設計されている。

5. **ガーベジコレクション（Garbage Collection）**:
   - メモリ管理の一種で、不要になったオブジェクトを自動的に検出し、メモリを解放するプロセス。特に大きなデータやモデルを扱う際にメモリ不足を防ぐために重要。

6. **注意マスク（Attention Mask）**:
   - ニューラルネットワークが入力データ中のどの部分を「見る」べきかを示すマスク。パディング（無視されるべき部分）を避け、モデルが有効な情報のみを処理することができる。

7. **スレッド（Thread）**:
   - プログラムが同時に実行できるプロセスの単位。スレッドを使用して、異なる処理を同時並行で行うことで、計算の効率を上げることができる。

8. **量子化（Quantization）**:
   - モデルの重みやデータを減少させる手法で、小さいビット数（例えば16ビットや4ビット）に変換することで、メモリ使用量と計算コストを削減する。これにより、モデルを軽量化し、特にGPUの性能を最大限に引き出すことができる。

9. **自動混合精度（autocast）**:
   - 計算の精度を動的に調整する技術。モデルの計算をフロート16（半精度）やフロート32（単精度）の間で切り替え、高速かつメモリ効率を改善する。

10. **トンネル効果（Truncation）**:
    - 入力が指定された最大長を超えた場合に、その超過部分を切り捨てる処理。通常、モデルが処理できる最大のトークン数に制限をかけるために行う。

これらの用語は、ノートブックに特有の文脈や実務経験に基づいたものであり、初心者が理解する上で役立つでしょう。

---


In [ ]:
# bitsandbytesライブラリをインストールします。
# --no-indexオプションは、PyPIリポジトリを使用せず、指定したファイルシステムのリンクからのみインストールします。
!pip install -q -U bitsandbytes --no-index --find-links ../input/libs-install

# transformersライブラリをインストールします。
# 同様に、--no-indexオプションを使用して、指定したリンクからのみインストールします。
!pip install -q -U transformers --no-index --find-links ../input/libs-install

In [ ]:
# 必要なライブラリをインポートします。
import os  # OSに関連する機能を提供するライブラリ
import gc  # ガーベジコレクタ（不要なオブジェクトをメモリから削除するためのライブラリ）
import re  # 正規表現を使用するためのライブラリ
from time import time  # 時間に関する機能を提供するライブラリ

import torch  # PyTorchライブラリ
import transformers  # Hugging Faceのtransformersライブラリ
import sklearn  # 機械学習のためのライブラリ
import random  # ランダムな数値を生成するためのライブラリ
import numpy as np  # 数値計算を行うためのライブラリ（NumPy）
import pandas as pd  # データ分析を行うためのライブラリ（Pandas）
import matplotlib.pyplot as plt  # グラフ描画を行うためのライブラリ

# モデルとトークナイザーをインポートします。
from transformers import Gemma2ForCausalLM, GemmaTokenizer, BitsAndBytesConfig

import time  # 再度timeライブラリをインポート（必要ない場合は削除可能）
from catboost import CatBoostClassifier, Pool  # CatBoostライブラリをインポート（決定木による分類器）
from sklearn.model_selection import train_test_split  # データを訓練セットとテストセットに分割する機能
from sklearn.metrics import accuracy_score, log_loss  # モデルの評価指標を計算するための機能

from torch.cuda.amp import autocast  # 自動混合精度を使用するための機能をインポート
from threading import Thread  # スレッドの管理を行うためのライブラリ

# CUDAのメモリアイデアを有効にする設定
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# GPUが使用可能か確認し、使用できない場合はメッセージを表示します。
if (not torch.cuda.is_available()):
    print("Sorry - GPU required!")  # GPUが必要であることを知らせるメッセージを表示します。

In [ ]:
# 訓練データフレームをCSVファイルから読み込みます。
train_df = pd.read_csv('/kaggle/input/embedding/train_embed.csv')

# 訓練データの埋め込み（特徴量）をNumpy配列として読み込みます。
train_embed = np.load('/kaggle/input/embedding/gemma2_train_embed.npy')

# 各行のラベルを設定します。
# winner_model_a, winner_model_b, winner_tie の中で最大の値を持つ列を見つけ、その列のインデックスをラベルとして使用します。
train_df.loc[:, 'label'] = np.argmax(train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].values, axis=1)  # どのモデルが勝つかを示すラベルを作成します。

In [ ]:
# ターゲット変数の列名を定義します。
Targets = ['winner_model_a', 'winner_model_b', 'winner_tie']

# 予測対象のラベルを取得します。
y = train_df['label'].values

# 訓練データとテストデータに分割します。
# train_test_split関数を使用して、インデックスを訓練用とテスト用に分けます。
train_idx, test_idx = train_test_split(train_df.index, test_size=0.1, random_state=42, stratify=y)

# 訓練データとテストデータの特徴量とラベルを設定します。
X_train, y_train = train_embed[train_idx], train_df.iloc[train_idx]['label'].values  # 訓練データ
X_test, y_test = train_embed[test_idx], train_df.iloc[test_idx]['label'].values  # テストデータ

# 訓練データとテストデータの形状を出力します。
print(X_train.shape, y_train.shape)  # 訓練データの形状を出力
print(X_test.shape, y_test.shape)      # テストデータの形状を出力

In [ ]:
# CatBoostClassifierのモデルを初期化します。
model_cb = CatBoostClassifier()

# 保存されたCatBoostモデルを読み込みます。
# 指定されたパスからモデルをロードします。
model_cb.load_model('/kaggle/input/catboost-mike/catboost.cbm')

In [ ]:
# CatBoostモデルの情報を表示します。
# これにより、モデルの構造やパラメータ、トレーニング状況などの詳細を確認することができます。
model_cb

In [ ]:
# モデルのパスと設定を定義します。
MODEL_PATH = '/kaggle/input/gemma-2-9b-hf'  # Gemmaモデルの保存先パス
MAX_LENGTH = 1024  # モデルが処理する最大のシーケンス長
BATCH_SIZE = 2  # バッチサイズ

# 使用するGPUデバイスを指定します。
device0 = torch.device('cuda:0')  # 1つ目のGPUデバイス
device1 = torch.device('cuda:1')  # 2つ目のGPUデバイス

# トークナイザーをモデルからロードします。
tokenizer = GemmaTokenizer.from_pretrained(MODEL_PATH)

# 量子化の設定を定義します。
bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビットモードで読み込みます。
    bnb_4bit_compute_dtype=torch.float16,  # 計算精度としてfloat16を使用します。
    bnb_4bit_use_double_quant=False)  # 二重量子化を使用しない設定

# 1つ目のGPUデバイスにモデルをロードします。
model_0 = Gemma2ForCausalLM.from_pretrained(MODEL_PATH,
                                        revision="float16",  # float16バージョンを使用
                                        device_map='cuda:0',  # GPUデバイス0にマッピング
                                        quantization_config=bnb_config_4bit)        

# 2つ目のGPUデバイスにモデルをロードします。
model_1 = Gemma2ForCausalLM.from_pretrained(MODEL_PATH,
                                        revision="float16",  # float16バージョンを使用
                                        device_map='cuda:1',  # GPUデバイス1にマッピング
                                        quantization_config=bnb_config_4bit)

In [ ]:
# 入力文字列を処理する関数を定義します。
def process(input_str):
    # 文字列の先頭と末尾のブラケットを削除します。
    stripped_str = input_str.strip('[]')
    # カンマで区切られた文をリストに分割し、前後の空白を削除します。
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # 文が存在する場合は最後の文を返し、存在しない場合は空文字を返します。
    return sentences[-1] if sentences else ''

# テストデータをCSVファイルから読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# 'prompt', 'response_a', 'response_b' 列に対してprocess関数を適用します。
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# テキスト列を構成します。モデルの応答を整理した形式です。
test['text'] = '<start_of_turn>User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n----\n\nModel B:\n'  + test['response_b'] + '<end_of_turn><eos>'

# 最初のテキストを出力します。
print(test['text'][0])  # 生成された最初のテキストの形式を表示します。

In [ ]:
# トークナイザーを使用してテキストをトークン化します。
tokens = tokenizer(test['text'].tolist(),
                   padding='max_length',  # 最大長にパディングを行います。
                   max_length=MAX_LENGTH,  # 定義した最大長を適用します。
                   truncation=True,  # 長さが超過する場合は切り捨てます。
                   return_tensors='pt')  # PyTorchテンソルとして返します。

# テキストデータをDataFrameに変換します。
data = pd.DataFrame()
# トークンの入力IDと注意マスクをリストとしてDataFrameに追加します。
data['INPUT_IDS'] = [tensor.tolist() for tensor in tokens['input_ids']]
data['ATTENTION_MASKS'] = [tensor.tolist() for tensor in tokens['attention_mask']]

# 最初の2行を表示します。
data[:2]  # データの最初の2行を表示して内容を確認します。

In [ ]:
# 埋め込みを取得する関数を定義します。
def get_embeddings(df, model, device, batch_size=BATCH_SIZE):  
    # INPUT_IDSとATTENTION_MASKSをテンソルに変換します。
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)

    embed_list = []  # 埋め込みのリストを初期化します。

    # バッチごとに処理を行います。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # 最後のバッチのインデックスを設定
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチの注意マスクをデバイスに転送
        
        # ガーベジコレクションを実施し、メモリをクリアします。
        gc.collect() 
        torch.cuda.empty_cache()

        # 勾配計算を行わずにモデルを通して埋め込みを取得します。
        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, output_hidden_states=True)
            embed = outputs.hidden_states[-1]  # 最後の隠れ層の出力を取得
            embed_mean = torch.mean(embed, dim=1).cpu()  # 平均プールを行い、CPUに戻します。
            embed_list.append(embed_mean)  # 得られた埋め込みをリストに追加
            
            torch.cuda.empty_cache()  # 再度メモリをクリア
        
    # リストの埋め込みを1つのテンソルに結合して返します。
    embeddings = torch.cat(embed_list, dim=0)
    return embeddings  # 取得した埋め込みを返します。

# 埋め込みを計算し結果を保存する関数を定義します。
def compute_embed(df, model, device, results, index):
    results[index] = get_embeddings(df, model, device)  # 指定されたインデックスに埋め込み結果を格納します。

In [ ]:
# 処理開始時刻を記録します。
st = time.time()

# データのサンプル数を取得します。
N_SAMPLES = len(data)
half = round(N_SAMPLES / 2)  # データを2つのサブセットに分けるためのインデックスを計算
sub1 = data.iloc[0:half].copy()  # 最初の半分のデータをコピー
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りの半分のデータをコピー

# 結果を格納する辞書を初期化します。
results = {}

# それぞれのスレッドを作成し、compute_embed関数を呼び出します。
t0 = Thread(target=compute_embed, args=(sub1, model_0, device0, results, 0))  # スレッド0
t1 = Thread(target=compute_embed, args=(sub2, model_1, device1, results, 1))  # スレッド1

# スレッドを開始します。
t0.start()
t1.start()

# 両方のスレッドが完了するのを待ちます。
t0.join()
t1.join()

# 処理の完了時刻を表示します。
print(f"Processing complete. Total time: {time.time() - st:.2f} seconds")  # 処理にかかった合計時間を表示します。

In [ ]:
# 結果の埋め込みを結合します。
# 先ほどのスレッドで計算した埋め込みを1つのテンソルにまとめます。
test_embeddings = torch.cat([results[0], results[1]], dim=0)

# 結合した埋め込みの形状を出力します。
test_embeddings.shape  # テンソルの形状を表示して、埋め込みのサイズを確認します。

In [ ]:
# ガーベジコレクションを実行して、不要なオブジェクトをメモリから削除します。
gc.collect()

# 使用し終わったモデルを削除します。
del model_1  # モデル1を削除
del model_0  # モデル0を削除

# GPUメモリをクリアします。
torch.cuda.empty_cache()  # 不要なメモリを解放します。

In [ ]:
# CatBoostモデルを使用して予測確率を計算します。
# テストデータの埋め込みに対して、モデルが各クラスが選ばれる確率を予測します。
preds = model_cb.predict_proba(test_embeddings.numpy())

# 予測結果を表示します。
preds  # 各モデルの応答が選ばれる確率の配列を出力します。

In [ ]:
# 提出用サンプルファイルをCSVから読み込みます。
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

# 予測結果をサンプル提出用データフレームに設定します。
sample_sub[Targets] = preds  # 予測確率をターゲット列に代入します。

# サンプル提出用データを表示します。
display(sample_sub)  # 最終的な提出用データフレームを画面に表示します。

In [ ]:
# 提出用データフレームをCSVファイルとして保存します。
sample_sub.to_csv('submission.csv', index=False)  # インデックスなしで'submission.csv'という名前で保存します。